In [1]:
!pip install google-cloud
!pip install google-cloud-pubsub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 10.2 MB/s eta 0:00:00


In [1]:
import os
import requests
import json
from time import sleep
from google.cloud import pubsub_v1

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'GOOGLE_APPLICATION_CREDENTIALS'

In [ ]:
project_id = "<project_id>"
# topic_id = "your-topic-id"
topic_id = "topico-vuelos"
publisher = pubsub_v1.PublisherClient()
project_path = f"projects/{project_id}"

In [ ]:
access_key = "<API-KEY-AVIATIONSTACK>"
limit = "5"
flight_status = "active"
arr_iata = "BCN"

aviation_url = "http://api.aviationstack.com/v1/flights" + \
            "?access_key=" + access_key + \
            "&flight_status=" + flight_status + \
            "&arr_iata=" + arr_iata
            # "&limit=" + limit

topic_path = publisher.topic_path(project_id, topic_id)

In [ ]:
while True:
    # Get all flights
    response = requests.get(aviation_url)
    if response.status_code != 200:
        print("Algo ocurrió. ERROR: ", response.status_code)
        print(response.json)
        exit()

    json_text = response.text

    python_obj = json.loads(json_text)

    data_array = []
    for flight in python_obj['data']:
        data = {}

        # Only flight in the air
        if flight['live'] != None:
            data['vuelo'] = flight['flight']['iata']
            data['aerolinea'] = flight['airline']['name']
            data['fecha_vuelo'] = flight['flight_date']
            data['origen'] = flight['departure']['airport']
            data['destino'] = flight['arrival']['airport']
            data['latitud'] = flight['live']['latitude']
            data['longitud'] = flight['live']['longitude']
            data['altitud'] = flight['live']['altitude']
            data['velocidad'] = flight['live']['speed_horizontal']
            data['timestamp'] = flight['live']['updated']
            print("Enviando información del vuelo " + data['vuelo'])
            data_array.append(data)

    print("\nEnviando array: ")
    print(data_array)

    future = publisher.publish(topic_path, json.dumps(data_array).encode("utf-8"))
    print(future.result())

    print("\n")

    sleep(5)

Enviando información del vuelo VY2017
Enviando información del vuelo FR2403
Enviando información del vuelo VY6503
Enviando información del vuelo VY6405
Enviando información del vuelo VY7333
Enviando información del vuelo FR6375
Enviando información del vuelo VY3209
Enviando información del vuelo VY1873
Enviando información del vuelo VY8103
Enviando información del vuelo TK1841
Enviando información del vuelo VY7839
Enviando información del vuelo VY1815
Enviando información del vuelo VL1816
Enviando información del vuelo VY2474
Enviando información del vuelo LH1138
Enviando información del vuelo FR2331
Enviando información del vuelo VY1887
Enviando información del vuelo VY8985
Enviando información del vuelo VY6249
Enviando información del vuelo VY8017
Enviando información del vuelo VY3003
Enviando información del vuelo VY8465
Enviando información del vuelo TP1040
Enviando información del vuelo AZ78
Enviando información del vuelo U27156
Enviando información del vuelo VY8241
Enviando infor

KeyboardInterrupt: 